In [ ]:
# Enable autoreload of module
%load_ext autoreload
%autoreload 2

In [115]:
from torchinfo import summary
import torch
import os

from data.DWSNets_dataset import DWSNetsDataset, LayerOneHotTransform, FlattenTransform, BiasFlagTransform
from networks.naive_rq_ae import RQAutoencoder, RQAutoencoderConfig

In [ ]:

dir_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(dir_path, "adl4cv", "datasets", "DWSNets", "mnist-inrs")

class AutoencoderTransform(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.bias = BiasFlagTransform()
    self.flatten = FlattenTransform()
    self.layer_one_hot = LayerOneHotTransform()

  def forward(self, x, y):
    bias = self.bias(x)
    layer = self.layer_one_hot(x)    
    x = self.flatten(x)
    return torch.hstack((x, layer, bias)), y



train_dataset = DWSNetsDataset(data_root, transform=AutoencoderTransform())
test_dataset = DWSNetsDataset(data_root, split="test", transform=AutoencoderTransform())
path = "datasets/DWSNets/mnist-inrs/mnist_splits.json"


In [117]:
ae_config = RQAutoencoderConfig(dim_l=(5, 64, 4, 3))
ae = RQAutoencoder(ae_config)

print(train_dataset[203][0])
print(ae(train_dataset[203][0]))

tensor([[ 0.0190,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0074,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0846,  1.0000,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.1998,  0.0000,  0.0000,  1.0000,  0.0000],
        [-0.1661,  0.0000,  0.0000,  1.0000,  0.0000],
        [-0.0389,  0.0000,  0.0000,  1.0000,  1.0000]])
tensor([[ 0.1465,  0.3759, -0.2415, -0.0327, -0.0401],
        [ 0.1465,  0.3759, -0.2415, -0.0327, -0.0401],
        [ 0.1465,  0.3759, -0.2415, -0.0327, -0.0401],
        ...,
        [ 0.1465,  0.3759, -0.2415, -0.0327, -0.0401],
        [ 0.1465,  0.3759, -0.2415, -0.0327, -0.0401],
        [ 0.1465,  0.3759, -0.2415, -0.0327, -0.0401]],
       grad_fn=<AddmmBackward0>)


In [128]:
from training.training_autoencoder import train_model, TrainingConfig

from torch.nn import MSELoss
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

train_model(TrainingConfig(), ae_config, train_dataloader, test_dataloader, MSELoss())

batch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
batch,100
epoch,0
loss,0.05036


KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.


In [120]:
torch.cuda.is_available()

False